In [8]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm

In [9]:
graph = "memoryalpha"
embeddings = "dogtag_bgem3"

In [10]:
embeddings_file = "./_input/node_embeddings/" + embeddings + "/" + graph + ".json"
url_embeddings_file = "./_input/url_embeddings/" + graph + ".json"
graph_file = "data/filtered_triples/" + graph + ".triples"
neighborhood_embeddings_file = "./_input/neighborhood_embeddings/" + embeddings + "/" + graph + ".json"

In [11]:
df = pd.read_json(embeddings_file)
df_T = df.T

In [12]:
df_url = pd.read_json(url_embeddings_file)
df_url_T = df_url.T

In [13]:
graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
G_nx = nx.Graph()
for row in graph_df.itertuples():
    G_nx.add_edge(int(row[1]), int(row[2]))
print(G_nx)

Graph with 69445 nodes and 1307028 edges


In [14]:
embeddings_dict = dict()

for n in tqdm(G_nx.nodes):
    
    neighbors = nx.neighbors(G_nx, int(n))
    embeddings = []

    try:
        selected_embedding = df_T.loc[int(n)]
        embeddings.append(selected_embedding)
    except KeyError:
        try:
            selected_embedding = df_url_T.loc[int(n)]
            embeddings.append(selected_embedding)
        except KeyError:
            print(str(int(n)) + " --- Key not found anywhere!")

    for node in neighbors:
        try:
            selected_embedding = df_T.loc[node]
            embeddings.append(selected_embedding)
        except KeyError:
            try:
                selected_embedding = df_url_T.loc[node]
                embeddings.append(selected_embedding)
            except KeyError:
                print(str(node) + " --- Key not found anywhere!")

    neighborhood_embedding = np.average(embeddings, axis=0)
    embeddings_dict.update({n: {str(j): val for j, val in enumerate(neighborhood_embedding)}})

with open(neighborhood_embeddings_file, "w") as json_file:
    json.dump(embeddings_dict, json_file, indent=4)

 18%|█▊        | 12528/69445 [00:34<01:06, 862.26it/s]

64312 --- Key not found anywhere!


 21%|██        | 14415/69445 [00:36<00:57, 953.03it/s] 

163342 --- Key not found anywhere!


 27%|██▋       | 18628/69445 [00:40<00:40, 1255.10it/s]

...

 91%|█████████▏| 63490/69445 [01:01<00:01, 3616.16it/s]

391157 --- Key not found anywhere!


 93%|█████████▎| 64926/69445 [01:02<00:01, 3541.21it/s]

373547 --- Key not found anywhere!


100%|██████████| 69445/69445 [01:03<00:00, 1092.87it/s]
